In [1]:
import sqlite3
import pandas as pd

In [2]:
# Connect to SQLite database (it will create one if it doesn't exist)
connection = sqlite3.connect('bank_transaction_analysis.db')
cursor = connection.cursor()


In [3]:
# Create Customers table
cursor.execute('''
CREATE TABLE IF NOT EXISTS Customers (
    customer_id INTEGER PRIMARY KEY,
    name TEXT NOT NULL,
    city TEXT,
    country TEXT,
    date_joined TEXT
);
''')

# Create Accounts table
cursor.execute('''
CREATE TABLE IF NOT EXISTS Accounts (
    account_id INTEGER PRIMARY KEY,
    customer_id INTEGER,
    account_type TEXT,
    balance DECIMAL(15, 2),
    FOREIGN KEY (customer_id) REFERENCES Customers(customer_id)
);
''')

# Create Transactions table
cursor.execute('''
CREATE TABLE IF NOT EXISTS Transactions (
    transaction_id INTEGER PRIMARY KEY,
    account_id INTEGER,
    transaction_date TEXT,
    transaction_type TEXT,
    amount DECIMAL(15, 2),
    status TEXT,
    FOREIGN KEY (account_id) REFERENCES Accounts(account_id)
);
''')

# Insert sample data
# Customers
cursor.execute("INSERT OR IGNORE INTO Customers VALUES (1, 'Alice Smith', 'New York', 'USA', '2020-01-01');")
cursor.execute("INSERT OR IGNORE INTO Customers VALUES (2, 'Bob Johnson', 'Los Angeles', 'USA', '2021-03-15');")
cursor.execute("INSERT OR IGNORE INTO Customers VALUES (3, 'Charlie Davis', 'London', 'UK', '2019-07-10');")
cursor.execute("INSERT OR IGNORE INTO Customers VALUES (4, 'Diana White', 'Toronto', 'Canada', '2022-04-25');")

# Accounts
cursor.execute("INSERT OR IGNORE INTO Accounts VALUES (1, 1, 'Savings', 5000.00);")
cursor.execute("INSERT OR IGNORE INTO Accounts VALUES (2, 1, 'Checking', 2000.00);")
cursor.execute("INSERT OR IGNORE INTO Accounts VALUES (3, 2, 'Savings', 10000.00);")
cursor.execute("INSERT OR IGNORE INTO Accounts VALUES (4, 3, 'Savings', 7500.00);")
cursor.execute("INSERT OR IGNORE INTO Accounts VALUES (5, 4, 'Checking', 1500.00);")

# Transactions
cursor.execute("INSERT OR IGNORE INTO Transactions VALUES (1, 1, '2024-01-15', 'Deposit', 1000.00, 'Completed');")
cursor.execute("INSERT OR IGNORE INTO Transactions VALUES (2, 1, '2024-01-20', 'Withdrawal', 500.00, 'Completed');")
cursor.execute("INSERT OR IGNORE INTO Transactions VALUES (3, 2, '2024-02-01', 'Payment', 200.00, 'Completed');")
cursor.execute("INSERT OR IGNORE INTO Transactions VALUES (4, 3, '2024-02-10', 'Transfer', 1500.00, 'Completed');")
cursor.execute("INSERT OR IGNORE INTO Transactions VALUES (5, 4, '2024-03-05', 'Deposit', 2000.00, 'Completed');")
cursor.execute("INSERT OR IGNORE INTO Transactions VALUES (6, 5, '2024-03-12', 'Withdrawal', 5000.00, 'Flagged');")

# Commit the changes
connection.commit()


In [4]:
query = '''
SELECT transaction_type, COUNT(*) AS transaction_count, SUM(amount) AS total_amount
FROM Transactions
GROUP BY transaction_type
ORDER BY total_amount DESC;
'''

result = pd.read_sql_query(query, connection)
result


,transaction_type,transaction_count,total_amount
0,Withdrawal,2,5500
1,Deposit,2,3000
2,Transfer,1,1500
3,Payment,1,200


In [5]:
query = '''
SELECT Transactions.transaction_id, Customers.name, Accounts.account_type, Transactions.amount, Transactions.status
FROM Transactions
JOIN Accounts ON Transactions.account_id = Accounts.account_id
JOIN Customers ON Accounts.customer_id = Customers.customer_id
WHERE Transactions.status = 'Flagged';
'''

result = pd.read_sql_query(query, connection)
result


,transaction_id,name,account_type,amount,status
0,6,Diana White,Checking,5000,Flagged


In [6]:
query = '''
SELECT Customers.name, strftime('%Y-%m', Transactions.transaction_date) AS month, 
       AVG(Transactions.amount) AS avg_monthly_spending
FROM Customers
JOIN Accounts ON Customers.customer_id = Accounts.customer_id
JOIN Transactions ON Accounts.account_id = Transactions.account_id
WHERE Transactions.transaction_type IN ('Withdrawal', 'Payment')
GROUP BY Customers.name, month
ORDER BY month;
'''

result = pd.read_sql_query(query, connection)
result


,name,month,avg_monthly_spending
0,Alice Smith,2024-01,500.0
1,Alice Smith,2024-02,200.0
2,Diana White,2024-03,5000.0


In [7]:
query = '''
SELECT Customers.name, SUM(Transactions.amount) AS total_deposit
FROM Customers
JOIN Accounts ON Customers.customer_id = Accounts.customer_id
JOIN Transactions ON Accounts.account_id = Transactions.account_id
WHERE Transactions.transaction_type = 'Deposit'
GROUP BY Customers.name
ORDER BY total_deposit DESC;
'''

result = pd.read_sql_query(query, connection)
result


,name,total_deposit
0,Charlie Davis,2000
1,Alice Smith,1000


In [8]:
connection.close()
